In [1]:
import pandas as pd

In [54]:
df = pd.read_csv("weatherAUS.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
melb_df = df[df["Location"]=="Melbourne"]

In [55]:
melb_df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
67200,2008-07-01,Melbourne,9.5,15.4,2.0,2.8,7.0,W,63.0,W,...,60.0,38.0,1006.8,1010.3,1.0,7.0,11.0,14.6,Yes,No
67201,2008-07-02,Melbourne,11.0,14.5,0.6,5.2,4.0,W,63.0,W,...,66.0,52.0,1013.4,1016.2,7.0,4.0,11.8,13.7,No,Yes
67202,2008-07-03,Melbourne,8.4,14.1,1.4,2.0,0.8,W,39.0,W,...,72.0,58.0,1024.8,1024.3,7.0,7.0,10.6,13.9,Yes,No
67203,2008-07-04,Melbourne,9.1,14.5,0.6,1.8,4.2,E,19.0,NNW,...,87.0,49.0,1030.2,1029.0,7.0,6.0,10.2,13.1,No,No
67204,2008-07-05,Melbourne,4.3,15.5,0.0,0.8,8.4,N,52.0,N,...,81.0,52.0,1028.9,1024.9,2.0,5.0,5.5,14.6,No,No


In [56]:
melb_df["Cloudiness"]=(melb_df["Cloud9am"]+melb_df["Cloud3pm"])/2/8*100
melb_df["Humidity"]=(melb_df["Humidity9am"]+melb_df["Humidity3pm"])/2
melb_df["Pressure"]=(melb_df["Pressure9am"]+melb_df["Pressure3pm"])/2

/opt/anaconda3/envs/MachineL/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/anaconda3/envs/MachineL/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/envs/MachineL/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

In [57]:
y = clean_melb_df["RainTomorrow"]

In [59]:
clean_melb_df

,MinTemp,MaxTemp,Rainfall,WindGustSpeed,RainToday,RainTomorrow,Cloudiness,Humidity,Pressure
67200,9.5,15.4,2.0,63.0,Yes,No,50.00,49.0,1008.55
67201,11.0,14.5,0.6,63.0,No,Yes,68.75,59.0,1014.80
67202,8.4,14.1,1.4,39.0,Yes,No,87.50,65.0,1024.55
67203,9.1,14.5,0.6,19.0,No,No,81.25,68.0,1029.60
67204,4.3,15.5,0.0,52.0,No,No,43.75,66.5,1026.90
...,...,...,...,...,...,...,...,...,...
70388,7.8,13.5,0.6,24.0,No,No,87.50,90.0,1028.60
70389,6.7,14.1,0.2,24.0,No,No,87.50,74.5,1026.50
70390,7.4,14.9,0.0,44.0,No,Yes,50.00,70.0,1016.30
70391,8.3,12.3,1.4,30.0,Yes,No,56.25,69.0,1018.05


In [60]:
clean_melb_df = melb_df
clean_melb_df["Month"]=clean_melb_df.apply(lambda x: int(x["Date"].split("-")[1]),axis=1)
clean_melb_df = melb_df.drop(["RainTomorrow","Date","Location","WindGustDir","Evaporation", "Sunshine","WindDir9am","WindDir3pm", "WindSpeed9am", "WindSpeed3pm", "Humidity9am", "Humidity3pm", "Cloud9am", "Cloud3pm","Pressure9am","Pressure3pm","Temp9am","Temp3pm"],axis=1)

/opt/anaconda3/envs/MachineL/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [66]:
clean_melb_df.RainToday = clean_melb_df.RainToday.eq('Yes').mul(1)

In [68]:
X = clean_melb_df

In [70]:
from sklearn.preprocessing import StandardScaler

# Create a StandardScater model and fit it to the training data

### BEGIN SOLUTION
X_scaler = StandardScaler().fit(X)

In [71]:
X_scaler

StandardScaler()

In [73]:
X_scaled = X_scaler.transform(X)
X_scaled[0]

array([-0.51803993, -0.86638071,  0.0169249 ,  1.08614478,  1.64310836,
       -0.64743525, -0.78342059, -1.0461279 ,  0.09809545])